In [1]:
from pymongo import MongoClient, TEXT, GEOSPHERE
from pprint import pprint

client = MongoClient()
db = client.geo
restaurants = db.restaurants
states = db.states

In [2]:
# create text index
restaurants.create_index([('name', TEXT)])

'name_text'

# Exercise 3-1

In [3]:
# TODO : find restaurants whose name include "Kimchi"
#       use text index
result = restaurants.find({'$text':{'$search':'Kimchi'}})

for item in result:
    print('zipcode: {}, address: {} {}'.format(
        item['address']['zipcode'], item['address']['street'], item['address']['building'])
    )

zipcode: 11231, address: Smith Street 478
zipcode: 11238, address: Washington Avenue 766
zipcode: 10036, address: West 48 Street 18-20


# Exercise 3-2

In [14]:
# TODO : find restaurants whose name include "Nolbu Restaurant" 
#       use text index
nolbu_coord = restaurants.find_one({'$text':{'$search':"Nolbu"}})

# find the state which has nolbu restaurant
name = states.find_one({
    'loc': {
        '$geoIntersects': {
            '$geometry': nolbu_coord['address']['coord']}}})['name']

print('--- nolbu ---')
pprint(nolbu_coord)

print('--- result ---')
print(name)

TypeError: 'NoneType' object is not subscriptable

# Exercise 3-3

In [16]:
#restaurants.create_index([('address.coord', GEOSPHERE)])

# TODO : among the korean restaurants, find a restaurant
#       which has "Ramen" but not "Izakaya" in its name.
#       use text index
r_obj = restaurants.find_one({'$text':{'$search':'Ramen -izakaya'}})

# make geo object with r_obj (type : Point)
geo_obj = {
    'type': 'Point',
    'coordinates': r_obj['address']['coord']
}

# find a Japanese restaurant near r_obj in 150m
#        use $near and $ne
result = restaurants.find({
    'cuisine': 'Japanese',
    'address.coord': {
        '$near': {
            '$geometry': geo_obj,
            '$maxDistance': 150
        }
    },
    'name': {
        '$ne':  r_obj['name']
    }
})

print('--- ramen ---')
pprint(r_obj)
print('--- near ---')
pprint(list(result))

--- ramen ---
{'_id': ObjectId('5d6c83705aaadf7838a8d4c7'),
 'address': {'building': '1574',
             'coord': [-73.9535392, 40.7750381],
             'street': '2 Avenue',
             'zipcode': '10028'},
 'borough': 'Manhattan',
 'cuisine': 'Other',
 'grades': [],
 'name': 'Ramen Meijin ',
 'restaurant_id': '50014784'}
--- near ---
[{'_id': ObjectId('5d6c83705aaadf7838a8b6dc'),
  'address': {'building': '1567',
              'coord': [-73.9541551, 40.7749987],
              'street': '2Nd Ave',
              'zipcode': '10028'},
  'borough': 'Manhattan',
  'cuisine': 'Japanese',
  'grades': [{'date': datetime.datetime(2014, 3, 24, 0, 0),
              'grade': 'A',
              'score': 10},
             {'date': datetime.datetime(2013, 10, 16, 0, 0),
              'grade': 'A',
              'score': 13},
             {'date': datetime.datetime(2013, 4, 9, 0, 0),
              'grade': 'A',
              'score': 12},
             {'date': datetime.datetime(2012, 8, 20, 0, 0),

In [7]:
client.close()